In [1]:
!pip install -U segmentation_models_pytorch timm
!pip install -q albumentations==1.4.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

## Библиотеки и очень полезные вещи

In [2]:
import os, time, random, math, numpy as np
import torch, torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
from torch.amp import GradScaler, autocast
from tqdm.auto import tqdm
from typing import Optional, Callable, Tuple

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

torch.use_deterministic_algorithms(True, warn_only=True)

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print('DEVICE:', DEVICE, torch.cuda.get_device_name(0) if DEVICE=='cuda' else '')

IMG_SIZE      = 320
BATCH_SIZE    = 8
ACCUM_STEPS   = 2
NUM_WORKERS   = 0
NUM_CLASSES   = 21
ROOT_DIR      = './data'

MEAN = (0.485, 0.456, 0.406)
STD  = (0.229, 0.224, 0.225)

DEVICE: cuda Tesla T4


## Аугментации

### Зачем такие аугментации?

- **RandomResizedCrop** помогает модели быть инвариантной к масштабу и фрейминигу объекта.  
- **HorizontalFlip** увеличивает разнообразие ориентаций без изменения семантики.  
- **ShiftScaleRotate** восстанавливает устойчивость к небольшим геометрическим искажениям (кадрирование, наклоны).  
- **ColorJitter** и **GaussianBlur** (в strong_aug) учат модель устойчивости к изменению освещения, контраста и шуму.  
- **Нормализация** стандартизирует входы под предобученные сети.  
- **Разделение train/val** с разными уровнями агрессии аугментаций предотвращает утечку артефактов в валидацию.

In [3]:
train_aug = A.Compose([
    A.RandomResizedCrop(
        height=IMG_SIZE, width=IMG_SIZE,
        scale=(0.5, 1.5), ratio=(0.75, 1.33)
    ),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(0.10, 0.15, 15, p=0.5),
    A.Normalize(MEAN, STD),
    ToTensorV2(),
])

val_aug = A.Compose([
    A.Resize(height=IMG_SIZE, width=IMG_SIZE),
    A.Normalize(MEAN, STD),
    ToTensorV2(),
])

strong_aug = A.Compose([
    A.RandomResizedCrop(
        height=IMG_SIZE, width=IMG_SIZE,
        scale=(0.4, 1.8)
    ),
    A.HorizontalFlip(p=0.5),
    A.ColorJitter(0.4, 0.4, 0.4, 0.2, p=0.5),
    A.ShiftScaleRotate(0.20, 0.25, 20, p=0.7),
    A.GaussianBlur(p=0.25),
    A.Normalize(MEAN, STD),
    ToTensorV2(),
])

<ipython-input-3-84950e3d1090>:2: DeprecationWarning: Initializing with 'height' and 'width' is deprecated. Please use a tuple (height, width) for the 'size' argument.
  A.RandomResizedCrop(
<ipython-input-3-84950e3d1090>:19: DeprecationWarning: Initializing with 'height' and 'width' is deprecated. Please use a tuple (height, width) for the 'size' argument.
  A.RandomResizedCrop(


## Враппер датасета

- **Источник:** обёртка над `torchvision.datasets.VOCSegmentation(year=2012)`
- **Transform:** принимает `image`+`mask` вместе (Albumentations) → гарантирует совпадение аугментаций  
- **Выход:**  
  - `img: torch.Tensor` (C×H×W, нормализованный)  
  - `mask: torch.LongTensor` (H×W, целые лейблы)  
- **Зачем:**  
  - Синхронные геометрические/цветовые аугментации  
  - Простой PyTorch-совместимый `Dataset` для сегментации 

In [4]:
class VOCSegDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        root: str,
        image_set: str,
        transform: Optional[Callable] = None,
    ):
        self.base = datasets.VOCSegmentation(
            root,
            year="2012",
            image_set=image_set,
            download=True,
        )
        self.transform = transform

    def __len__(self) -> int:
        return len(self.base)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        img, mask = self.base[idx]
        mask = np.array(mask, dtype=np.int64)

        if self.transform:
            sample = self.transform(
                image=np.array(img),
                mask=mask,
            )
            img = sample["image"]
            mask = sample["mask"].long()

        return img, mask

## Загрузчики данных

Почему выбрали PASCAL VOC 2012 для сегментации

- **Стандартный бенчмарк:** широко используется в научных работах по семантической сегментации.  
- **Разнообразие классов:** 20 категорий + фон, подходит для многоклассовой постановки.  
- **Достаточный объём:** ~10 000 изображений — позволяет обучать и валидировать модели без чрезмерных затрат.  
- **Готовые маски:** разметка «pixel-wise» облегчает оценку метрик (mIoU, Jaccard). 

In [5]:
gen = torch.Generator().manual_seed(SEED)

train_full = VOCSegDataset(ROOT_DIR, 'train', transform=train_aug)
n_total = len(train_full)
n_train = int(0.9 * n_total)
n_val = n_total - n_train
train_ds, val_ds = random_split(train_full, [n_train, n_val], generator=gen)
val_ds.dataset.transform = val_aug

test_ds = VOCSegDataset(ROOT_DIR, 'val', transform=val_aug)

print(f'Dataset sizes ➜ train {len(train_ds)} | val {len(val_ds)} | test {len(test_ds)}')

train_loader = DataLoader(train_ds, BATCH_SIZE, shuffle=True,
                          generator=gen, num_workers=NUM_WORKERS, pin_memory=True)
val_loader = DataLoader(val_ds,   BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=True)
test_loader = DataLoader(test_ds,  BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=True)

100%|██████████| 2.00G/2.00G [02:01<00:00, 16.5MB/s]


Dataset sizes ➜ train 1317 | val 147 | test 1449


## Полезные штуки

- **IoU (Intersection over Union)**  
  Оценивает, насколько область предсказанной маски пересекается с областью истинной маски для каждого класса. Чем выше, тем лучше совпадение.

- **mIoU (mean IoU)**  
  Среднее значение IoU по всем классам (кроме пикселей с меткой `ignore_index`). Позволяет сравнивать качество сегментации независимо от частоты классов.

In [6]:
def mean_iou(
    logits: torch.Tensor,
    masks: torch.Tensor,
    ignore: int = 255,
) -> float:
    preds = logits.argmax(dim=1).cpu().numpy()
    gts = masks.cpu().numpy()

    ious: list[float] = []

    for cls in range(NUM_CLASSES):
        if cls == ignore:
            continue

        pred_count = (preds == cls).sum()
        gt_count = (gts == cls).sum()
        if pred_count == 0 and gt_count == 0:
            continue

        inter = np.logical_and(preds == cls, gts == cls).sum()
        union = np.logical_or(preds == cls, gts == cls).sum()

        if union == 0:
            continue

        ious.append(inter / union)

    return float(np.mean(ious)) if ious else 0.0

def fit_model(
    model: nn.Module,
    epochs: int,
    lr: float,
    train_aug_pipe: A.Compose,
) -> float:
    train_ds.dataset.transform = train_aug_pipe
    criterion = nn.CrossEntropyLoss(ignore_index=255)
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=lr,
        weight_decay=1e-4,
    )
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer,
        T_max=epochs,
    )
    scaler = GradScaler()

    best_miou = 0.0
    best_state: dict[str, torch.Tensor] | None = None

    for epoch in range(1, epochs + 1):
        model.train()
        optimizer.zero_grad()
        epoch_loss = 0.0

        for step, (x, y) in enumerate(tqdm(train_loader, leave=False), start=1):
            x = x.to(DEVICE)
            y = y.to(DEVICE)

            with autocast(device_type=DEVICE):
                logits = model(x)
                loss = criterion(logits, y) / ACCUM_STEPS

            scaler.scale(loss).backward()

            if step % ACCUM_STEPS == 0 or step == len(train_loader):
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

            epoch_loss += loss.item() * ACCUM_STEPS

        scheduler.step()

        model.eval()
        val_ious: list[float] = []

        with torch.no_grad():
            for x, y in val_loader:
                x = x.to(DEVICE)
                with autocast(device_type=DEVICE):
                    val_ious.append(mean_iou(model(x), y))

        miou = float(np.mean(val_ious))
        avg_loss = epoch_loss / len(train_loader)

        print(
            f"E{epoch:02d}/{epochs}  "
            f"loss {avg_loss:.3f}  "
            f"val mIoU {miou:.3f}"
        )

        if miou > best_miou:
            best_miou = miou
            best_state = model.state_dict()

    if best_state is not None:
        model.load_state_dict(best_state)

    return best_miou

def evaluate(
    model: nn.Module,
    name: str,
) -> float:
    model.eval()
    test_ious: list[float] = []

    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(DEVICE)
            with autocast(device_type=DEVICE):
                test_ious.append(mean_iou(model(x), y))

    miou = float(np.mean(test_ious))

    print(f"{name:16s} ➜ test mIoU {miou:.3f}")
    return miou

## Baseline

### U-Net, но не U-Net

- **Архитектура**: стандартный UNet с энкодером ResNet-34, предобученным на ImageNet; выход без активации (`activation=None`).  
- **Обучение**: 8 эпох, lr=1e-4, базовая аугментация (случайный кроп, флип, поворот, нормализация), батч-накопление = 1.

In [7]:
unet34 = smp.Unet('resnet34', encoder_weights='imagenet',
                  classes=NUM_CLASSES, activation=None).to(DEVICE)
fit_model(unet34, epochs=8, lr=1e-4, train_aug_pipe=train_aug)
miou_unet34 = evaluate(unet34, 'UNet-R34')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

  0%|          | 0/165 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:3494: UserWarning: nll_loss2d_forward_out_cuda_template does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:92.)
  return torch._C._nn.cross_entropy_loss(


...

E06/8  loss 1.083  val mIoU 0.159


  0%|          | 0/165 [00:00<?, ?it/s]

E07/8  loss 1.024  val mIoU 0.170


  0%|          | 0/165 [00:00<?, ?it/s]

E08/8  loss 0.996  val mIoU 0.163
UNet-R34         ➜ test mIoU 0.156


- Loss стабильно падает, мIoU растёт до ~0.17 к 7-й эпохе, затем немного снижается → лёгкое переобучение.  
- Тестовая mIoU (0.156) задаёт отправную точку для сравнения с улучшенными и трансформерными моделями.  

### Трансформер

- **Архитектура**: SegFormer с энкодером MiT-B0 (легковесный трансформер), выход без активации.  
- **Обучение**: 8 эпох, lr=3e-4, базовая аугментация (`train_aug`), батч-накопление = 1

In [8]:
segformer = smp.Segformer('mit_b0', encoder_weights='imagenet',
                          classes=NUM_CLASSES, activation=None).to(DEVICE)
fit_model(segformer, epochs=8, lr=3e-4, train_aug_pipe=train_aug)
miou_segformer = evaluate(segformer, 'SegFormer-B0')

config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/14.3M [00:00<?, ?B/s]

  0%|          | 0/165 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/segmentation_models_pytorch/encoders/mix_transformer.py:153: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:217.)
  attn = (q @ k.transpose(-2, -1)) * self.scale
/usr/local/lib/python3.11/dist-packages/segmentation_models_pytorch/encoders/mix_transformer.py:157: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(tr

E01/8  loss 1.533  val mIoU 0.175


...

E06/8  loss 0.306  val mIoU 0.414


  0%|          | 0/165 [00:00<?, ?it/s]

E07/8  loss 0.271  val mIoU 0.417


  0%|          | 0/165 [00:00<?, ?it/s]

E08/8  loss 0.263  val mIoU 0.421
SegFormer-B0     ➜ test mIoU 0.407


- Модель быстро учится: mIoU растёт особенно активно в первые 6 эпох.  
- Дальнейшее улучшение замедляется → стоило бы остановиться примерно на 7 эпохе.  
- Тестовый mIoU (0.407) в ~2.6× выше, чем у UNet-R34 (0.156) — сильное преимущество трансформерного бэкбона для сегментации.

## Крутые baseline

### U-Net ultra super +

- **Архитектура**: UNet ++ с энкодером EfficientNet-B3 (ImageNet), выход без активации.  
- **Обучение**: 10 эпох, lr=1e-4, используется «strong_aug» (агрессивные аугментации).

In [9]:
unetpp = smp.UnetPlusPlus('efficientnet-b3', encoder_weights='imagenet',
                          classes=NUM_CLASSES, activation=None).to(DEVICE)
fit_model(unetpp, epochs=10, lr=1e-4, train_aug_pipe=strong_aug)
miou_unetpp = evaluate(unetpp, 'UNet++-B3')

config.json:   0%|          | 0.00/106 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/49.3M [00:00<?, ?B/s]

  0%|          | 0/165 [00:00<?, ?it/s]

E01/10  loss 2.780  val mIoU 0.047


  0%|          | 0/165 [00:00<?, ?it/s]

...

E08/10  loss 1.143  val mIoU 0.052


  0%|          | 0/165 [00:00<?, ?it/s]

E09/10  loss 1.149  val mIoU 0.053


  0%|          | 0/165 [00:00<?, ?it/s]

E10/10  loss 1.155  val mIoU 0.053
UNet++-B3        ➜ test mIoU 0.059


- Несмотря на снижение loss, mIoU застрял около 0.05 → модель учится «вхолостую».  
- Агрессивные преобразования, вероятно, делают задачу слишком сложной.  
- Требуются более щадящие аугментации или настройка гиперпараметров, чтобы извлечь пользу из сильной модели. 

### Трансформер, но как автобот

- **Архитектура**: DeepLabV3+ с энкодером `timm-mobilenetv3_large_100` (ImageNet), без активации на выходе.  
- **Обучение**: 12 эпох, lr = 5e-4, используется «strong_aug» (агрессивные аугментации).

In [10]:
deeplab = smp.DeepLabV3Plus('timm-mobilenetv3_large_100',
                            encoder_weights='imagenet',
                            classes=NUM_CLASSES, activation=None).to(DEVICE)
fit_model(deeplab, epochs=12, lr=5e-4, train_aug_pipe=strong_aug)
miou_deeplab = evaluate(deeplab, 'DeepLabV3+')

model.safetensors:   0%|          | 0.00/22.1M [00:00<?, ?B/s]

  0%|          | 0/165 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/conv.py:549: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:217.)
  return F.conv2d(


E01/12  loss 1.286  val mIoU 0.266


  0%|          | 0/165 [00:00<?, ?it/s]

...

E10/12  loss 0.296  val mIoU 0.415


  0%|          | 0/165 [00:00<?, ?it/s]

E11/12  loss 0.297  val mIoU 0.388


  0%|          | 0/165 [00:00<?, ?it/s]

E12/12  loss 0.298  val mIoU 0.407
DeepLabV3+       ➜ test mIoU 0.421


- mIoU активно растёт до ~0.42 к 10-й эпохе, затем слегка флуктирует → настройка lr и аугментаций удачна.  
- Лёгкий энкодер MobileNetV3 позволяет получить высокую точность при небольшом размере модели.  
- DeepLabV3+ демонстрирует лучший тестовый mIoU в сравнении с другими архитектурами на этом этапе.  

## Свои модели

### Обычная

- **Архитектура**: компактный U-Net с базовым числом каналов = 32, два сверточных слоя в каждом блоке, апсемплинг через билинейную интерполяцию и суммирование со скип-коннекшнами.  
- **Обучение**: 8 эпох, lr = 3e-4, аугментации `train_aug`.

In [11]:
class TinyUNet(nn.Module):
    def __init__(self, n_cls=NUM_CLASSES, base=32):
        super().__init__()
        self.d1 = self._block(3, base)
        self.d2 = self._block(base, base*2)
        self.d3 = self._block(base*2, base*4)
        self.pool = nn.MaxPool2d(2,2)
        self.bot = self._block(base*4, base*8)
        self.u3 = self._up(base*8, base*4)
        self.u2 = self._up(base*4, base*2)
        self.u1 = self._up(base*2, base)
        self.head = nn.Conv2d(base, n_cls, 1)
    def _block(self, c_in, c_out):
        return nn.Sequential(
            nn.Conv2d(c_in, c_out, 3, padding=1), nn.BatchNorm2d(c_out), nn.ReLU(inplace=True),
            nn.Conv2d(c_out, c_out, 3, padding=1), nn.BatchNorm2d(c_out), nn.ReLU(inplace=True)
        )
    def _up(self, c_in, c_out):
        return nn.Sequential(nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
                             nn.Conv2d(c_in, c_out, 3, padding=1), nn.ReLU(inplace=True))
    def forward(self, x):
        d1 = self.d1(x)
        d2 = self.d2(self.pool(d1))
        d3 = self.d3(self.pool(d2))
        b  = self.bot(self.pool(d3))
        x  = self.u3(b) + d3
        x  = self.u2(x) + d2
        x  = self.u1(x) + d1
        return self.head(x)

tiny = TinyUNet().to(DEVICE)
fit_model(tiny, epochs=8, lr=3e-4, train_aug_pipe=train_aug)
miou_tiny = evaluate(tiny, 'TinyUNet')

  0%|          | 0/165 [00:00<?, ?it/s]

E01/8  loss 1.595  val mIoU 0.066


  0%|          | 0/165 [00:00<?, ?it/s]

E02/8  loss 1.452  val mIoU 0.066


  0%|          | 0/165 [00:00<?, ?it/s]

...

E06/8  loss 1.354  val mIoU 0.069


  0%|          | 0/165 [00:00<?, ?it/s]

E07/8  loss 1.342  val mIoU 0.069


  0%|          | 0/165 [00:00<?, ?it/s]

E08/8  loss 1.324  val mIoU 0.068
TinyUNet         ➜ test mIoU 0.069


- mIoU практически не растёт выше 0.07 → модель слишком лёгкая для сегментации VOC.  
- Простая архитектура без достаточной пропускной способности неспособна захватить сложные объекты.  
- TinyUNet полезен как эталон «минимальной» модели, но для реальной задачи нужны более мощные архитектуры.  

### Супер плюс плюс своя модель

- **Архитектура**: наследник TinyUNet, добавлен `Dropout2d(p=0.2)` на выходе для регуляризации.  
- **Обучение**: 10 эпох, lr = 1e-3, аугментации `strong_aug`

In [12]:
class TinyUNetPlus(TinyUNet):
    def __init__(self):
        super().__init__()
        self.drop = nn.Dropout2d(p=0.2)
    def forward(self, x):
        x = super().forward(x)
        return self.drop(x)

tiny_plus = TinyUNetPlus().to(DEVICE)
fit_model(tiny_plus, epochs=10, lr=1e-3, train_aug_pipe=strong_aug)
miou_tiny_plus = evaluate(tiny_plus, 'TinyUNet+')

  0%|          | 0/165 [00:00<?, ?it/s]

E01/10  loss 1.912  val mIoU 0.067


  0%|          | 0/165 [00:00<?, ?it/s]

E02/10  loss 1.825  val mIoU 0.070


  0%|          | 0/165 [00:00<?, ?it/s]

...

E08/10  loss 1.734  val mIoU 0.067


  0%|          | 0/165 [00:00<?, ?it/s]

E09/10  loss 1.720  val mIoU 0.068


  0%|          | 0/165 [00:00<?, ?it/s]

E10/10  loss 1.716  val mIoU 0.067
TinyUNet+        ➜ test mIoU 0.070


- Добавление Dropout слегка выровняло результаты и дало небольшой прирост на валидации.  
- Общий прирост по сравнению с TinyUNet минимален (0.070 vs 0.069), модель по-прежнему слишком лёгкая для задачи VOC.  
- Dropout помогает бороться с переобучением, но для эффективной сегментации нужны более глубокие и сложные архитектуры.

## Результаты

In [13]:
scores = {
    'UNet-R34'   : miou_unet34,
    'SegFormer-B0': miou_segformer,
    'UNet++-B3'  : miou_unetpp,
    'DeepLabV3+' : miou_deeplab,
    'TinyUNet'   : miou_tiny,
    'TinyUNet+'  : miou_tiny_plus,
}

print('\n### Итоговая mIoU')
for n,v in scores.items():
    print(f'{n:15s}: {v:.3f}')

base_name = 'UNet-R34'
best_name = max(scores, key=scores.get)
gain = 100 * (scores[best_name]-scores[base_name]) / scores[base_name]
print(f'\nЛучшая модель — {best_name} (mIoU {scores[best_name]:.3f}), '
      f'превышение над бейзлайном {base_name}: {gain:.1f}%')


### Итоговая mIoU
UNet-R34       : 0.156
SegFormer-B0   : 0.407
UNet++-B3      : 0.059
DeepLabV3+     : 0.421
TinyUNet       : 0.069
TinyUNet+      : 0.070

Лучшая модель — DeepLabV3+ (mIoU 0.421), превышение над бейзлайном UNet-R34: 169.4%


- **DeepLabV3+ (0.421)** вновь показывает лидирующий результат благодаря эффективному ASPP-блок-механизму и лёгкому, но выразительному энкодеру MobileNetV3.  
- **SegFormer-B0 (0.407)** почти дотягивает до DeepLabV3+ — трансформер хорошо справляется с глобальным контекстом даже в младшем варианете.  
- **UNet-R34 (0.156)**, как базовый CNN-подход, демонстрирует скромный результат: простая «склеивающая» архитектура плохо обрабатывает сложные границы сегментов.  
- **UNet++-B3 (0.059)** оказался значительно слабее — вложенные мостики и больший энкодер не компенсировали недостаток адекватных аугментаций и более жёсткую оптимизацию.  
- **TinyUNet / TinyUNet+ (0.069 / 0.070)** — собственные упрощённые U-сети слишком мелки, чтобы захватывать необходимые признаки; добавление дропаут-блока почти не изменило ситуацию.  

В целом, глубокие архитектуры со специализированными блоками (ASPP в DeepLabV3+, многоуровневые представления в SegFormer) гораздо эффективнее классического U-Net на VOC2012. 